# Calculate cometary coma colors in LSST filters from gas and dust production rates

Generate model coma fluxes using the Afρ model for dust and the Haser model for gas.

## Requirements

Recent version of `sbpy`, `astropy`.  Some kind of `matplotlib` for figures.

In [1]:
# pip install git+https://git@github.com/NASA-Planetary-Science/sbpy.git#egg=sbpy

In [2]:
import numpy as np
import astropy.units as u
from astropy.io import ascii
from astropy.table import Table
from sbpy.activity import CircularAperture
from sbpy.data import Ephem
import sbpy.units as sbu
from sbpy.spectroscopy import SpectralGradient
from lsst_cometary_colors import Comet, lambda_eff

## Gas reference data

Scalelengths, band emission g-factors, and throughput for LSST filters.

In [3]:
ascii.read("scalelengths.csv", delimiter=",").show_in_notebook()

idx,molecule,gamma parent,gamma daughter,gamma reference
0,NH,50000.0,150000.0,Randall et al. (1993)
1,CN,13000.0,210000.0,Randall et al. (1993)
2,C3,2800.0,27000.0,Randall et al. (1993)
3,C2,22000.0,66000.0,Randall et al. (1993)
4,NH2,4900.0,62000.0,Fink et al. (1991)


In [4]:
ascii.read("emission-bands.csv", delimiter=",").show_in_notebook()

idx,molecule,band,wave,gfactor,reference,u,g,r,i,z,y
0,NH,(0-0),0.01,3358.0,Kim et al. (1989),1.0,0.0,0.0,0.0,0.0,0.0
1,CN,(Delta-nu=0),3870.0,0.078,Schleicher (1983),1.0,1.0,0.0,0.0,0.0,0.0
2,CN,(2-0),--,nan,Fink (1994),0.0,0.0,0.0,1.0,0.0,0.0
3,CN,(1-0),--,nan,Fink (1994),0.0,0.0,0.0,0.0,1.0,1.0
4,C3,(4050),4050.0,0.2,A'Hearn et al. (1985),1.0,1.0,0.0,0.0,0.0,0.0
5,C2,(Delta-nu=1),nan,nan,--,0.0,1.0,0.0,0.0,0.0,0.0
6,C2,(Delta-nu=0),5150.0,0.12,A'Hearn (1982),0.0,1.0,1.0,0.0,0.0,0.0
7,C2,(Delta-nu=-1),nan,nan,--,0.0,0.0,1.0,0.0,0.0,0.0
8,NH2,(3-0) Sigma,8159.0,0.00021,Tegler and Wyckoff (1989),0.0,0.0,0.0,1.0,1.0,0.0
9,NH2,(4-0) Pi,7935.0,0.00024,Tegler and Wyckoff (1989),0.0,0.0,0.0,1.0,1.0,0.0


## Coma models

The coma is described by a set of abundances.

The main observable in LSST data will be dust Afρ.  So, let's try to define the gas production rates with respect to this quantity.

To make things a little bit easier, let's define the gases with respect to CN, and then get the absolute production rate of CN based on a Q(CN)/Afρ value.

Why not water?  Just in case we need to test a wide range of heliocentric distances, but don't want unrealistic water production rates.

In [5]:
help(Comet)

Help on class Comet in module lsst_cometary_colors:

class Comet(builtins.object)
 |  Comet(eph, a0frho, S_V=<Quantity 0.1 1 / 100 nm>, QCN_afrho=<Quantity 2.1e+23 1 / (cm s)>, **gas_ratios)
 |  
 |  Comet = gas + dust.
 |  
 |  
 |  Parameters
 |  ----------
 |  eph : sbpy Ephem
 |      rh, delta, and phase.
 |  
 |  a0frho : astropy Quantity
 |      Dust production rate proxy at 0 deg phase angle at 0.55 μm, units of
 |      length.
 |  
 |  S_V : astropy Quantity, inverse length
 |      Spectral gradient at 0.55 μm (V-band).
 |  
 |  QCN_afrho : astropy Quantity
 |      Ratio of CN production rate to Afρ value.  Default is from A'Hearn et al.
 |      (1995):  CN / OH * OH / Afρ  = 10**(-2.50 + 25.82) = 2.1e23 cm / s.
 |  
 |  **gas_ratios :
 |      Additional keyword arguments are passed to ``Gas()``, e.g., NH_CN, C3_CN,
 |      C2_CN, and/or NH2_CN.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, eph, a0frho, S_V=<Quantity 0.1 1 / 100 nm>, QCN_afrho=<Quantity 2.1e+23 1 / (cm

In [6]:
eph = Ephem.from_horizons('C/2019 L3')
rho = CircularAperture(2 * u.arcsec).as_length(eph)
comet = Comet(eph, 1000 * u.cm)
gmr = comet.m('g', rho) - comet.m('r', rho)
gmr_dust = comet.dust.m('g', rho) - comet.dust.m('r', rho)
print(f'gas + dust: g-r = {gmr}')
print(f'dust only: g-r = {gmr_dust}')
print('Fractional contributions to g:')
for component, f in comet.fraction('g', rho).items():
    print(f'  {component}: {f}')
print('Fractional contributions to r:')
for component, f in comet.fraction('r', rho).items():
    print(f'  {component}: {f}')

gas + dust: g-r = [-0.04447034] mag
dust only: g-r = [0.57042126] mag
Fractional contributions to g:
  dust: 0.22029507540894053
  gas: 0.7797049245910596
  CN: {'total': 0.36966398158782593, '(Delta-nu=0)': 0.36966398158782593}
  C3: {'total': 0.034538722517437746, '(4050)': 0.034538722517437746}
  C2: {'total': 0.3084178851168721, '(Delta-nu=0)': 0.3084178851168721}
  NH2: {'total': 0.06708433536892373, '(10-0) Pi': 0.02445302458427069, '(11-0) Sigma': 0.01923945519177524, '(12-0) Pi': 0.011211481082976938, '(13-0) Sigma': 0.007012942899639895, '(14-0) Pi': 0.003552609232054421, '(15-0) Sigma': 0.0016148223782065552}
Fractional contributions to r:
  dust: 0.38811589298817545
  gas: 0.6118841070118246
  C2: {'total': 0.41649418597634735, '(Delta-nu=0)': 0.41649418597634735}
  NH2: {'total': 0.19538992103547728, '(6-0) Pi': 0.011464204550550965, '(7-0) Sigma': 0.02535832202214262, '(8-0) Pi': 0.030841202459362657, '(9-0) Sigma': 0.03738327570831836, '(9-0) Delta': 0.031339646135473555,